In [1]:
import torch
import torch.nn as nn
import torch.nn.modules.activation as activation
import math
import sys
sys.path.append('../model')  
from model import ExpActivation, Unsqueeze

### ConvNetDeep

In [2]:
c1 = nn.Conv1d(4, 100, 19)
bn1 = nn.BatchNorm1d(100)
rl1 = activation.ReLU()
mp1 = nn.MaxPool1d(3, 3)

c2 = nn.Conv1d(100, 200, 11)
bn2 = nn.BatchNorm1d(200)
rl2 = activation.ReLU()
mp2 = nn.MaxPool1d(3, 3)

c3 = nn.Conv1d(200, 200, 7)
bn3 = nn.BatchNorm1d(200)
rl3 = activation.ReLU()
mp3 = nn.MaxPool1d(4, 4) 

c7 = nn.Conv1d(200, 400, 4)
bn7 = nn.BatchNorm1d(400)
rl7 = activation.ReLU()
mp7 = nn.MaxPool1d(4, 4) 

# Block 4 : Fully Connected 1 :
d4 = nn.Linear(800, 800)  # 1000 for 200 input size
bn4 = nn.BatchNorm1d(800, 1e-05, 0.1, True)
rl4 = activation.ReLU()
dr4 = nn.Dropout(0.3)

# Block 5 : Fully Connected 2 :
d5 = nn.Linear(800, 800)
bn5 = nn.BatchNorm1d(800, 1e-05, 0.1, True)
rl5 = activation.ReLU()
dr5 = nn.Dropout(0.3)

# Block 6 :4Fully connected 3
num_classes = 1
d6 = nn.Linear(800, num_classes)
# self.sig = activation.Sigmoid()

In [23]:
x = torch.randn(10,4,608)
x = rl1(bn1(c1(x)))
print(f"Layer 1: {x.shape}")
x = mp1(x)
print(f"Layer 1: {x.shape}")


x = rl2(bn2(c2(x)))
print(f"Layer 2: {x.shape}")
x = mp2(x)
print(f"Layer 2: {x.shape}")


x = rl3(bn3(c3(x)))
print(f"Layer 3: {x.shape}")
x = mp3(x)
print(f"Layer 3: {x.shape}")

x = rl7(bn7(c7(x)))
print(f"Layer 3.5: {x.shape}")
x = mp7(x)
print(f"Layer 3.5: {x.shape}")

o = torch.flatten(x, start_dim=1)
print(f"o shape is {o.shape}")
o = rl4(bn4(d4(o)))
print(f"Layer 4: {o.shape}")
o = dr4(o)
print(f"Layer 4: {o.shape}")


o = rl5(bn5(d5(o)))
print(f"Layer 5: {o.shape}")
o = dr5(o)
print(f"Layer 5: {o.shape}")


o = d6(o)
print(f"Finalz: {o.shape}")


Layer 1: torch.Size([10, 100, 590])
Layer 1: torch.Size([10, 100, 196])
Layer 2: torch.Size([10, 200, 186])
Layer 2: torch.Size([10, 200, 62])
Layer 3: torch.Size([10, 200, 56])
Layer 3: torch.Size([10, 200, 14])
Layer 3.5: torch.Size([10, 400, 11])
Layer 3.5: torch.Size([10, 400, 2])
o shape is torch.Size([10, 800])
Layer 4: torch.Size([10, 800])
Layer 4: torch.Size([10, 800])
Layer 5: torch.Size([10, 800])
Layer 5: torch.Size([10, 800])
Finalz: torch.Size([10, 1])


In [13]:
11%2

1

### DanQ

In [2]:
class DanQ(nn.Module):
    """
    PyTorch implementation of DanQ (PMID: 27084946)
    """
    def __init__(self, input_length, num_classes, weight_path=None):
        """
        :param input_length: int, input sequence length
        :param num_classes: int, number of output classes
        :param weight_path: string, path to the file with model weights
        """
        super(DanQ, self).__init__()

        self._options = {
            "input_length": input_length,
            "num_classes": num_classes,
            "weight_path": weight_path
        }

        self.conv1 = nn.Conv1d(4, 320, kernel_size=26)
        self.act1 = nn.ReLU()
        self.maxp1 = nn.MaxPool1d(kernel_size=13, stride=13)

        self.bi_lstm_layer = nn.LSTM(320, 320, num_layers=1,
                                     batch_first=True, bidirectional=True)

        self._in_features_L1 = math.floor((input_length - 25) / 13.) * 640

        self.linear = nn.Sequential(
            nn.Linear(self._in_features_L1, 925),
            nn.ReLU(),
            nn.Linear(925, num_classes),
        )

        if weight_path:
            self.load_state_dict(torch.load(weight_path))

    def forward(self, input):      
        x = self.act1(self.conv1(input))
        x = nn.Dropout(0.2)(self.maxp1(x))
        x = x.transpose(1, 2)
        x, _ = self.bi_lstm_layer(x)
        x = x.contiguous().view(-1, self._in_features_L1)
        x = nn.Dropout(0.5)(x)
        x = self.linear(x)
        return x

In [5]:
# Usage of the DanQ model
input_length = 608
num_classes = 1

# Create an instance of the model
model = DanQ(input_length, num_classes)

# Prepare input tensor (10 samples, 4 channels, input_length of 608)
x = torch.randn(10, 4, input_length)

# Forward pass to get the output
output = model(x)

# Print the output shape
print(output.shape)

torch.Size([10, 1])


### ExplaiNN

In [5]:
class ExplaiNN(nn.Module):
    """
    The ExplaiNN model (PMID: 37370113)
    """
    def __init__(self, num_cnns, input_length, num_classes, filter_size=19, num_fc=2, pool_size=7, pool_stride=7,
                 weight_path=None):
        """
        :param num_cnns: int, number of independent cnn units
        :param input_length: int, input sequence length
        :param num_classes: int, number of outputs
        :param filter_size: int, size of the unit's filter, default=19
        :param num_fc: int, number of FC layers in the unit, default=2
        :param pool_size: int, size of the unit's maxpooling layer, default=7
        :param pool_stride: int, stride of the unit's maxpooling layer, default=7
        :param weight_path: string, path to the file with model weights
        """
        super(ExplaiNN, self).__init__()

        self._options = {
            "num_cnns": num_cnns,
            "input_length": input_length,
            "num_classes": num_classes,
            "filter_size": filter_size,
            "num_fc": num_fc,
            "pool_size": pool_size,
            "pool_stride": pool_stride,
            "weight_path": weight_path
        }

        if num_fc == 0:
            self.linears = nn.Sequential(
                nn.Conv1d(in_channels=4 * num_cnns, out_channels=1 * num_cnns, kernel_size=filter_size,
                          groups=num_cnns),
                nn.BatchNorm1d(num_cnns),
                ExpActivation(),
                nn.MaxPool1d(input_length - (filter_size-1)),
                nn.Flatten())
        elif num_fc == 1:
            self.linears = nn.Sequential(
                nn.Conv1d(in_channels=4 * num_cnns, out_channels=1 * num_cnns, kernel_size=filter_size,
                          groups=num_cnns),
                nn.BatchNorm1d(num_cnns),
                ExpActivation(),
                nn.MaxPool1d(pool_size, pool_stride),
                nn.Flatten(),
                Unsqueeze(),
                nn.Conv1d(in_channels=int(((input_length - (filter_size-1)) - (pool_size-1)-1)/pool_stride + 1) * num_cnns,
                          out_channels=1 * num_cnns, kernel_size=1,
                          groups=num_cnns),
                nn.BatchNorm1d(1 * num_cnns, 1e-05, 0.1, True),
                nn.ReLU(),
                nn.Flatten())
        elif num_fc == 2:
            self.linears = nn.Sequential(
                nn.Conv1d(in_channels=4 * num_cnns, out_channels=1 * num_cnns, kernel_size=filter_size,
                          groups=num_cnns),
                nn.BatchNorm1d(num_cnns),
                ExpActivation(),
                nn.MaxPool1d(pool_size, pool_stride),
                nn.Flatten(),
                Unsqueeze(),
                nn.Conv1d(in_channels=int(((input_length - (filter_size-1)) - (pool_size-1)-1)/pool_stride + 1) * num_cnns,
                          out_channels=100 * num_cnns, kernel_size=1,
                          groups=num_cnns),
                nn.BatchNorm1d(100 * num_cnns, 1e-05, 0.1, True),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Conv1d(in_channels=100 * num_cnns,
                          out_channels=1 * num_cnns, kernel_size=1,
                          groups=num_cnns),
                nn.BatchNorm1d(1 * num_cnns, 1e-05, 0.1, True),
                nn.ReLU(),
                nn.Flatten())
        else:
            self.linears = nn.Sequential(
                nn.Conv1d(in_channels=4 * num_cnns, out_channels=1 * num_cnns, kernel_size=filter_size,
                          groups=num_cnns),
                nn.BatchNorm1d(num_cnns),
                ExpActivation(),
                nn.MaxPool1d(pool_size, pool_stride),
                nn.Flatten(),
                Unsqueeze(),
                nn.Conv1d(in_channels=int(((input_length - (filter_size-1)) - (pool_size-1)-1)/pool_stride + 1) * num_cnns,
                          out_channels=100 * num_cnns, kernel_size=1,
                          groups=num_cnns),
                nn.BatchNorm1d(100 * num_cnns, 1e-05, 0.1, True),
                nn.ReLU())

            self.linears_bg = nn.ModuleList([nn.Sequential(nn.Dropout(0.3),
                                                           nn.Conv1d(in_channels=100 * num_cnns,
                                                                     out_channels=100 * num_cnns, kernel_size=1,
                                                                     groups=num_cnns),
                                                           nn.BatchNorm1d(100 * num_cnns, 1e-05, 0.1, True),
                                                           nn.ReLU()) for i in range(num_fc - 2)])

            self.last_linear = nn.Sequential(nn.Dropout(0.3),
                                             nn.Conv1d(in_channels=100 * num_cnns, out_channels=1 * num_cnns,
                                                       kernel_size=1,
                                                       groups=num_cnns),
                                             nn.BatchNorm1d(1 * num_cnns, 1e-05, 0.1, True),
                                             nn.ReLU(),
                                             nn.Flatten())

        self.final = nn.Linear(num_cnns, num_classes)

        if weight_path:
            self.load_state_dict(torch.load(weight_path))

    def forward(self, x):
        x = x.repeat(1, self._options["num_cnns"], 1)
        print(f"x.repeat {x.shape}")
        if self._options["num_fc"] <= 2:
            outs = self.linears(x)
        else:
            outs = self.linears(x)
            for i in range(len(self.linears_bg)):
                outs = self.linears_bg[i](outs)
            outs = self.last_linear(outs)
        print(f"outs {outs.shape}")
        out = self.final(outs)
        print(f"out {out.shape}")
        return out


In [8]:
for learning_rate in [1e-5, 5e-5]:
    print(str(learning_rate)[:2] + str(learning_rate)[-1])

1e5
5e5


In [7]:
# Usage of the DanQ model
input_length = 608
num_classes = 1

model = ExplaiNN(num_cnns = 5, input_length = 608, num_classes = 2)

# Prepare input tensor (10 samples, 4 channels, input_length of 608)
x = torch.randn(10, 4, input_length)

# Forward pass to get the output
output = model(x)

# Print the output shape
print(output.shape)

x.repeat torch.Size([10, 20, 608])
outs torch.Size([10, 5])
out torch.Size([10, 2])
torch.Size([10, 2])
